In [282]:
import mysql.connector
import math 
db = mysql.connector.connect(
host = "localhost",
user = "root",
password ="",
database = "db"
)
cur = db.cursor()

In [283]:
#cur.execute("CREATE TABLE INVESTMENT( STOCK_NAME VARCHAR( 20 ) PRIMARY KEY , QUANTITY INT , BUY_PRICE FLOAT , CURRENT_TOTAL FLOAT , TOTAL_BUY FLOAT , TOTAL_SELL FLOAT)")

In [ ]:
def main( ):
    v = [];
    print('\t\t\t--------------------WELCOME TO INVESTMENT BOOK--------------------\n')
    while(True):
        print("\nENTER YOUR CHOICE :\n1.See Portfolio\n2.Show Weightage\n3.Investment insight\n4.Buy Stock\n5.Sell Stock\n6.Exit\n")
        choice = int( input( ) ) 
        v.sort( reverse = True )
        if choice == 1 :
            cur.execute( "SELECT * FROM INVESTMENT ORDER BY CURRENT_TOTAL DESC" )
            curr = cur.fetchall( )
            if( len( curr ) == 0 ) :
                print( "NO STOCK AVAILABLE" )
            else:
                print( "\nStock_Name\tQuantity\tBuy_Value\tCurrent_Total\tTotal Buy\t Total Sell\n" )
                for x in curr :
                    if x[ 1 ] > 0 :
                        print( x[ 0 ] , ' '*( 14 - len( x[ 0 ] ) ) , x[ 1 ] , ' ' * 12 , x[ 2 ] , ' ' * 11 , round( x[ 3 ] , 3 ) , ' ' * 8 , x[ 4 ] , ' ' * 10 , x[ 5 ] ) 
        elif choice == 2 :
            cur.execute( "SELECT * FROM INVESTMENT ORDER BY CURRENT_TOTAL DESC" )
            arr = cur.fetchall( )
            cnt = 0 
            for x in arr :
                if x[ 1 ] > 0 : 
                    cnt += 1 
            if cnt == 0 : 
                print( 'NO STOCK AVAILABLE')
            else:
                for x in arr :
                    print( x[ 0 ] , x[ 1 ] )
        elif( choice == 3 ):
            cur.execute( "SELECT SUM( Total_Buy ) FROM INVESTMENT " )
            Sum = cur.fetchone( )
            if( Sum[ 0 ] == None ):
                print( "EMPTY PORTFOLIO" ) 
            else :
                print( "TOTAL INVESTMENT :" , Sum[ 0 ] )
            cur.execute( "SELECT SUM( Total_Sell ) FROM INVESTMENT " )
            liquidity = cur.fetchone( )
            print( "TOTAL LIQUIDITY  :" , liquidity[ 0 ] )
            cur.execute( " SELECT SUM( current_total ) FROM INVESTMENT " )
            current_total = cur.fetchone( )
            print( "CURRENT PORTFOLIO:", current_total[ 0 ] )
            print( "NET PROFIT/LOSS  :" , liquidity[ 0 ] - ( Sum[ 0 ] - current_total[ 0 ] ) )
        elif( choice == 4 ): 
            m = input("\nWANT TO BUY NEW STOCK PRESS 0 , FOR EXSISTING STOCK PRESS 1 \n" )
            if int( m ) == 1 :
                cur.execute("SELECT * FROM INVESTMENT " )
                arr = cur.fetchall( )
                if len( arr ) == 0 :
                    print( "NO STOCK IN PORTFOLIO " )
                else:
                    print( "WHICH STOCK YOU WANT TO BUY :\n" )
                    for i , x in zip( range( len( arr ) ) , arr ):
                        print( i + 1 , x[ 0 ] )
                    t = int( input( ) )
                    if t > len(arr):
                        print( "INVALID OUTPUT")
                    else:
                        stock_name = arr[ t - 1 ][ 0 ]
                        quantity = arr[ t - 1 ][ 1 ]
                        buy_price = arr[ t - 1 ][ 2 ] 
                        avg_price = arr[ t - 1 ][ 3 ]
                        total_buy = arr[ t - 1 ][ 4 ]
                        q = float( input( "\nENTER QUANTITY\n" ) ) 
                        price = float( input( "\nENTER BUY PRICE\n" ) )
                        avg_price = ( avg_price + q * price ) / 2 
                        sql = f'UPDATE INVESTMENT SET CURRENT_TOTAL = "{avg_price}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( )
                        quantity += q 
                        sql = f'UPDATE INVESTMENT SET QUANTITY = "{quantity}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( )
                        total_buy += ( q * price )  
                        sql = f'UPDATE INVESTMENT SET TOTAL_BUY = "{total_buy}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( )
                        print( 'BOUGHT SUCCESSFULLY') 
            else:
                stock_name = input("\nENTER STOCK NAME:\n" ) 
                stock_name = stock_name.upper( )
                quantity = input( "\nENTER Quantity\n" ) 
                price = input( ("\nENTER BUY PRICE\n" ) )
                value = float( q ) * float( price )  
                sql = f'INSERT INTO INVESTMENT VALUES( "{stock_name}" , "{quantity}" , "{price}" , "{value}" , "{value}" , 0 )'
                cur.execute( sql ) 
                db.commit( )
        elif choice == 5:
            cur.execute( " SELECT * FROM INVESTMENT " )
            arr = cur.fetchall()
            if( len( arr ) == 0 ):
                print( "\nNO STOCK IN PORTFOLIO\n")
            else:
                print("\nWHICH STOCK YOU WANT TO SELL\n")
                for i , x in zip( range( len( arr ) ) , arr ):
                    print( i + 1 , x )
                t = int( input( ) )
                if t > len( arr ):
                    print('\t\t\t--------------------WRONG INPUT-------------------')
                else:
                    stock_name = arr[ t - 1 ][ 0 ]
                    quantity = arr[ t - 1 ][ 1 ]
                    buy_price = arr[ t - 1 ][ 2 ] 
                    current_value = arr[ t - 1 ][ 3 ]
                    total_sell = arr[ t - 1 ][ 5 ]
                    q = int( input('ENTER QUANTITY') )
                    p = float( input('ENTER PRICE') )
                    value = q * p 
                    total_sell += value 
                    if( q <= quantity) :
                        quantity -= q 
                        total_sell += value 
                        current_value -= q * buy_price
                        sql = f'UPDATE INVESTMENT SET QUANTITY = "{quantity}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( ) 
                        sql = f'UPDATE INVESTMENT SET TOTAL_SELL = "{total_sell}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( )
                        sql = f'UPDATE INVESTMENT SET CURRENT_TOTAL = "{current_value}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( )
                        print( 'SOLD SUCCESSFULLY' )
                    else:
                        print( '\t\t\t--------------------INVALID INPUT-----------------')
        elif choice == 6 :
            break 
        else:
            print("galat chal rha hai bhsdk")
if __name__ == "__main__":
    main()
